# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# Step 1.1: Read tic-tac-toe.csv into a dataframe

import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:


# Read the CSV file
df = pd.read_csv('tic-tac-toe.csv')

# Display basic information about the dataset
print("Dataset loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

# Display the first few rows
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Display column names
print("\nColumn names:")
print(df.columns.tolist())

# Display data types
print("\nData types:")
print(df.dtypes)

Dataset loaded successfully!

Dataset shape: (958, 10)
Number of rows: 958
Number of columns: 10

First 5 rows of the dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Column names:
['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']

Data types:
TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object


In [ ]:
# ============================================================================
# Step 1.2: Inspect the dataset - Check for data quality
# ============================================================================

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Check unique values in each column
print("\nUnique values in each column:")
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

# Check class distribution
print("\nClass distribution:")
print(df['class'].value_counts())


Missing values per column:
TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

Unique values in each column:
TL: ['x' 'o' 'b']
TM: ['x' 'o' 'b']
TR: ['x' 'o' 'b']
ML: ['x' 'o' 'b']
MM: ['o' 'b' 'x']
MR: ['o' 'b' 'x']
BL: ['x' 'o' 'b']
BM: ['o' 'x' 'b']
BR: ['o' 'x' 'b']
class: [ True False]

Class distribution:
class
True     626
False    332
Name: count, dtype: int64


In [ ]:
# ============================================================================
# Step 1.3: Convert categorical values to numeric in all columns
# ============================================================================

# Create a copy to preserve original data
df_numeric = df.copy()

# Define mapping for board positions
# x = 1, o = 0, b (blank) = -1
position_mapping = {'x': 1, 'o': 0, 'b': -1}

# Convert the first 9 columns (board positions) using the mapping
board_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
for col in board_columns:
    df_numeric[col] = df_numeric[col].map(position_mapping)

# Convert class column (True/False or positive/negative to 1/0)
# True (Player X wins) = 1, False (Player X doesn't win) = 0
df_numeric['class'] = df_numeric['class'].astype(int)

print("\nCategorical mapping applied:")
print(f"Board positions: {position_mapping}")
print(f"Class: True → 1, False → 0")

print("\nFirst 5 rows after conversion:")
print(df_numeric.head())

print("\nData types after conversion:")
print(df_numeric.dtypes)


Categorical mapping applied:
Board positions: {'x': 1, 'o': 0, 'b': -1}
Class: True → 1, False → 0

First 5 rows after conversion:
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   1   1   1   1   0   0   1   0   0      1
1   1   1   1   1   0   0   0   1   0      1
2   1   1   1   1   0   0   0   0   1      1
3   1   1   1   1   0   0   0  -1  -1      1
4   1   1   1   1   0   0  -1   0  -1      1

Data types after conversion:
TL       int64
TM       int64
TR       int64
ML       int64
MM       int64
MR       int64
BL       int64
BM       int64
BR       int64
class    int64
dtype: object


In [ ]:
# ============================================================================
# Step 1.4: Separate inputs and output
# ============================================================================

# Separate features (X) and target (y)
X = df_numeric[board_columns].values  # Input features (9 board positions)
y = df_numeric['class'].values        # Output (game result)

print(f"\nInput shape (X): {X.shape}")
print(f"Output shape (y): {y.shape}")

print(f"\nFirst 3 samples of X (inputs):")
print(X[:3])

print(f"\nFirst 3 samples of y (outputs):")
print(y[:3])


Input shape (X): (958, 9)
Output shape (y): (958,)

First 3 samples of X (inputs):
[[1 1 1 1 0 0 1 0 0]
 [1 1 1 1 0 0 0 1 0]
 [1 1 1 1 0 0 0 0 1]]

First 3 samples of y (outputs):
[1 1 1]


In [ ]:
# ============================================================================
# Step 1.5: Normalize the input data
# ============================================================================

# Normalize X from [-1, 1] to [0, 1] using Min-Max scaling
# Formula: X_normalized = (X - X_min) / (X_max - X_min)
X_min = X.min()
X_max = X.max()

X_normalized = (X - X_min) / (X_max - X_min)

print(f"\nOriginal data range: [{X_min}, {X_max}]")
print(f"Normalized data range: [{X_normalized.min()}, {X_normalized.max()}]")

print(f"\nFirst 3 samples of X_normalized:")
print(X_normalized[:3])

print(f"\nSummary:")
print(f"  • Total samples: {X_normalized.shape[0]}")
print(f"  • Input features: {X_normalized.shape[1]}")
print(f"  • Positive cases (X wins): {(y == 1).sum()}")
print(f"  • Negative cases (X loses): {(y == 0).sum()}")
print(f"\nData is ready for neural network training!")


Original data range: [-1, 1]
Normalized data range: [0.0, 1.0]

First 3 samples of X_normalized:
[[1.  1.  1.  1.  0.5 0.5 1.  0.5 0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 1.  0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 0.5 1. ]]

Summary:
  • Total samples: 958
  • Input features: 9
  • Positive cases (X wins): 626
  • Negative cases (X loses): 332

Data is ready for neural network training!


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# STEP 2: BUILD NEURAL NETWORK

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import numpy as np


# ============================================================================
# Step 2.1: Split the training and test data
# ============================================================================

# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Ensures equal distribution of classes
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"\nTraining set shape - X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Test set shape - X_test: {X_test.shape}, y_test: {y_test.shape}")

# Check class distribution
print(f"\nTraining set class distribution:")
print(f"  Class 1 (X wins): {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)")
print(f"  Class 0 (X loses/draw): {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# ----------------------------------------------------------------------------
# Verificar formato actual de las etiquetas
# ----------------------------------------------------------------------------
print("\nFormato ANTES de la conversión:")
print("-"*60)
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Ejemplo de y_train (primeras 5): {y_train[:5]}")
print(f"Valores únicos en y_train: {np.unique(y_train)}")

# ----------------------------------------------------------------------------
# Convertir a one-hot encoding
# ----------------------------------------------------------------------------
print("\nConvirtiendo a one-hot encoding...")
print("-"*60)

# Convertir y_train y y_test a one-hot encoding
# Si tus clases son 0, 1, 2 -> se convertirán a:
# 0 -> [1, 0, 0]  (X wins)
# 1 -> [0, 1, 0]  (O wins)
# 2 -> [0, 0, 1]  (Draw)

y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

print("✓ Conversión completada")


Formato ANTES de la conversión:
------------------------------------------------------------
y_train shape: (766,)
y_test shape: (192,)
Ejemplo de y_train (primeras 5): [1 0 1 0 1]
Valores únicos en y_train: [0 1]

Convirtiendo a one-hot encoding...
------------------------------------------------------------
✓ Conversión completada


In [ ]:
# ============================================================================
# PASO 2: CREAR MODELO 
# ============================================================================

# Crear un modelo completamente nuevo
model_new = Sequential()

# Agregar capas
model_new.add(Input(shape=(9,)))
model_new.add(Dense(units=64, activation='relu', name='hidden_layer_1'))
model_new.add(Dense(units=32, activation='relu', name='hidden_layer_2'))
model_new.add(Dense(units=3, activation='softmax', name='output_layer'))

print("✓ Modelo nuevo creado")

# Compilar el modelo
model_new.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_new.summary()



✓ Modelo nuevo creado


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,819 (11.01 KB)

 Trainable params: 2,819 (11.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ============================================================================
# PASO 3: ENTRENAR EL MODELO NUEVO
# ============================================================================

history = model_new.fit(
    X_train,
    y_train_encoded,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

print("\n✓ Entrenamiento completado!")



Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.5408 - loss: 0.8559 - val_accuracy: 0.6364 - val_loss: 0.7175
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6585 - loss: 0.6684 - val_accuracy: 0.6299 - val_loss: 0.6620
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6699 - loss: 0.6270 - val_accuracy: 0.6299 - val_loss: 0.6471
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6863 - loss: 0.6079 - val_accuracy: 0.6623 - val_loss: 0.6331
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6895 - loss: 0.5960 - val_accuracy: 0.6688 - val_loss: 0.6308
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7026 - loss: 0.5879 - val_accuracy: 0.6623 - val_loss: 0.6305
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7157 - loss: 0.5804 - val_accuracy: 0.6753 - val_loss: 0.6334
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7059 - loss: 0.5772 - val_accuracy: 0.6623 - v

In [ ]:
# ============================================================================
# PASO 4: EVALUAR EL MODELO
# ============================================================================

test_loss, test_accuracy = model_new.evaluate(
    X_test,
    y_test_encoded,
    verbose=0
)

print(f"✓ Test Loss: {test_loss:.4f}")
print(f"✓ Test Accuracy: {test_accuracy*100:.2f}%")

# ============================================================================
# PASO 5: GUARDAR EL MODELO
# ============================================================================
print("\n" + "="*60)
print("PASO 5: GUARDAR EL MODELO")
print("="*60)

model_new.save('tic-tac-toe.keras')
print("✓ Model saved as 'tic-tac-toe.keras'")

model_new.save('tic-tac-toe.h5')
print("✓ Model also saved as 'tic-tac-toe.h5'")

# ============================================================================
# RESUMEN FINAL
# ============================================================================
print("\n" + "="*60)
print("✓ TODO COMPLETADO EXITOSAMENTE!")
print("="*60)
print(f"\n📊 Resumen:")
print(f"  • Arquitectura: Sequential")
print(f"  • Capas: Input + 2 Hidden + Output (3 clases)")
print(f"  • Training samples: {X_train.shape[0]}")
print(f"  • Test samples: {X_test.shape[0]}")
print(f"  • Test Accuracy: {test_accuracy*100:.2f}%")
print(f"  • Modelo guardado: tic-tac-toe.keras")
print(f"\n🎉 ¡Modelo listo para hacer predicciones de Tic-Tac-Toe!")
print(f"\nClases:")
print(f"  0 = X wins")
print(f"  1 = O wins")  
print(f"  2 = Draw")

✓ Test Loss: 0.4083
✓ Test Accuracy: 80.21%

PASO 5: GUARDAR EL MODELO
✓ Model saved as 'tic-tac-toe.keras'
✓ Model also saved as 'tic-tac-toe.h5'

✓ TODO COMPLETADO EXITOSAMENTE!

📊 Resumen:
  • Arquitectura: Sequential
  • Capas: Input + 2 Hidden + Output (3 clases)
  • Training samples: 766
  • Test samples: 192
  • Test Accuracy: 80.21%
  • Modelo guardado: tic-tac-toe.keras

🎉 ¡Modelo listo para hacer predicciones de Tic-Tac-Toe!

Clases:
  0 = X wins
  1 = O wins
  2 = Draw


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# ============================================================================
# STEP 3: MAKE PREDICTIONS
# ============================================================================
print("\n" + "="*60)
print("STEP 3: LOAD MODEL AND MAKE PREDICTIONS")
print("="*60)

# ----------------------------------------------------------------------------
# Step 3.1: Load the saved model
# ----------------------------------------------------------------------------
print("\nStep 3.1: Loading the saved model")
print("-"*60)

from tensorflow.keras.models import load_model
import numpy as np

# Cargar el modelo guardado
loaded_model = load_model('tic-tac-toe.keras')
print("✓ Model loaded successfully from 'tic-tac-toe.keras'")

# Verificar la arquitectura
print("\nModel Summary:")
loaded_model.summary()

# ----------------------------------------------------------------------------
# Step 3.2: Select random samples from test dataset
# ----------------------------------------------------------------------------
print("\nStep 3.2: Selecting random samples from test dataset")
print("-"*60)

# Seleccionar 5 muestras aleatorias del conjunto de prueba
num_samples = 5
random_indices = np.random.choice(X_test.shape[0], size=num_samples, replace=False)

print(f"✓ Selected {num_samples} random samples from test set")
print(f"Random indices: {random_indices}")

# ----------------------------------------------------------------------------
# Step 3.3: Make predictions
# ----------------------------------------------------------------------------
print("\nStep 3.3: Making predictions")
print("-"*60)

# Obtener las muestras aleatorias
X_samples = X_test[random_indices]
y_samples = y_test_encoded[random_indices]  # Usar y_test_encoded

# Hacer predicciones (el modelo devuelve probabilidades para cada clase)
predictions = loaded_model.predict(X_samples, verbose=0)

# Convertir probabilidades a clases predichas
predicted_classes = np.argmax(predictions, axis=1)

# Convertir etiquetas verdaderas a clases
true_classes = np.argmax(y_samples, axis=1)

# Nombres de las clases
class_names = ['X wins', 'O wins', 'Draw']

print("✓ Predictions generated")

# ----------------------------------------------------------------------------
# Step 3.4: Display results
# ----------------------------------------------------------------------------
print("\nStep 3.4: Prediction Results")
print("-"*60)
print("\n")

for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"  Input features: {X_samples[i]}")
    print(f"  Probabilities:")
    print(f"    - X wins: {predictions[i][0]*100:.2f}%")
    print(f"    - O wins: {predictions[i][1]*100:.2f}%")
    print(f"    - Draw:   {predictions[i][2]*100:.2f}%")
    print(f"  Predicted class: {class_names[predicted_classes[i]]} (class {predicted_classes[i]})")
    print(f"  True class:      {class_names[true_classes[i]]} (class {true_classes[i]})")
    
    # Verificar si la predicción es correcta
    if predicted_classes[i] == true_classes[i]:
        print(f"  Result: ✓ CORRECT")
    else:
        print(f"  Result: ✗ INCORRECT")
    print("-"*60)

# ----------------------------------------------------------------------------
# Step 3.5: Summary
# ----------------------------------------------------------------------------
print("\nStep 3.5: Prediction Summary")
print("-"*60)

correct_predictions = np.sum(predicted_classes == true_classes)
accuracy = (correct_predictions / num_samples) * 100

print(f"\nCorrect predictions: {correct_predictions}/{num_samples}")
print(f"Accuracy on random samples: {accuracy:.2f}%")

# Mostrar distribución de predicciones
print("\nPrediction distribution:")
for i, class_name in enumerate(class_names):
    count = np.sum(predicted_classes == i)
    print(f"  {class_name}: {count}/{num_samples}")

print("\n" + "="*60)
print("✓ STEP 3 COMPLETE: PREDICTIONS MADE AND VERIFIED!")
print("="*60)
print("\n🎯 Your model successfully predicted Tic-Tac-Toe outcomes!")
print(f"   Model accuracy on these samples: {accuracy:.2f}%")


STEP 3: LOAD MODEL AND MAKE PREDICTIONS

Step 3.1: Loading the saved model
------------------------------------------------------------
✓ Model loaded successfully from 'tic-tac-toe.keras'

Model Summary:


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,459 (33.05 KB)

 Trainable params: 2,819 (11.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,640 (22.04 KB)


Step 3.2: Selecting random samples from test dataset
------------------------------------------------------------
✓ Selected 5 random samples from test set
Random indices: [ 69  66  17 112  10]

Step 3.3: Making predictions
------------------------------------------------------------
✓ Predictions generated

Step 3.4: Prediction Results
------------------------------------------------------------


Sample 1:
  Input features: [0.  0.  1.  1.  0.  1.  0.5 0.5 0.5]
  Probabilities:
    - X wins: 88.09%
    - O wins: 11.85%
    - Draw:   0.05%
  Predicted class: X wins (class 0)
  True class:      X wins (class 0)
  Result: ✓ CORRECT
------------------------------------------------------------
Sample 2:
  Input features: [1.  0.5 1.  0.5 1.  0.5 0.5 1.  1. ]
  Probabilities:
    - X wins: 9.00%
    - O wins: 91.00%
    - Draw:   0.00%
  Predicted class: O wins (class 1)
  True class:      O wins (class 1)
  Result: ✓ CORRECT
------------------------------------------------------------
Sa

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here